In [1]:
#!pip install -U langgraph langchain_openai --quiet

In [2]:
# !pip uninstall -y google-adk
#!pip install git+https://github.com/google/adk-python.git

## Import Data

In [1]:
import google.adk.tools.mcp_tool.mcp_toolset
import google.adk.tools.mcp_tool
import google.adk.tools.mcp_tool.mcp_session_manager

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Applied Agentic AI/HW5')
import database_setup

In [4]:
from database_setup import DatabaseSetup

# 1. Create DB helper pointing to "support.db"
db = DatabaseSetup("support.db")

# 2. Connect
db.connect()

# 3. Create tables (customers, tickets)
db.create_tables()

# 4. Create triggers (for updated_at)
db.create_triggers()

# 5. Insert sample data (customers + tickets)
db.insert_sample_data()

# 6. (Optional) Show schema + a few records
db.display_schema()
db.run_sample_queries()  # or comment out if too verbose

# 7. Close connection
db.close()


Connected to database: support.db
Tables created successfully!
Triggers created successfully!
Sample data inserted successfully!
  - 15 customers added
  - 25 tickets added

DATABASE SCHEMA

CUSTOMERS TABLE:
------------------------------------------------------------
  id              INTEGER     
  name            TEXT       NOT NULL 
  email           TEXT        
  phone           TEXT        
  status          TEXT       NOT NULL DEFAULT 'active'
  created_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP
  updated_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP

TICKETS TABLE:
------------------------------------------------------------
  id              INTEGER     
  customer_id     INTEGER    NOT NULL 
  issue           TEXT       NOT NULL 
  status          TEXT       NOT NULL DEFAULT 'open'
  priority        TEXT       NOT NULL DEFAULT 'medium'
  created_at      DATETIME    DEFAULT CURRENT_TIMESTAMP

FOREIGN KEYS:
------------------------------------------------------------
  tick

In [5]:
import sqlite3

conn = sqlite3.connect("support.db")
cur = conn.cursor()

cur.execute("SELECT id, name, email, status FROM customers LIMIT 5;")
print(cur.fetchall())

cur.execute("SELECT id, customer_id, issue, status, priority FROM tickets LIMIT 5;")
print(cur.fetchall())

conn.close()

[(1, 'John Doe', 'john.doe@example.com', 'active'), (2, 'Jane Smith', 'jane.smith@example.com', 'active'), (3, 'Bob Johnson', 'bob.johnson@example.com', 'disabled'), (4, 'Alice Williams', 'alice.w@techcorp.com', 'active'), (5, 'Charlie Brown', 'charlie.brown@email.com', 'active')]
[(1, 1, 'Cannot login to account', 'open', 'high'), (2, 4, 'Database connection timeout errors', 'in_progress', 'high'), (3, 7, 'Payment processing failing for all transactions', 'open', 'high'), (4, 10, 'Critical security vulnerability found', 'in_progress', 'high'), (5, 14, 'Website completely down', 'resolved', 'high')]


## Configuration

In [6]:
# # # Install required packages
%pip install --upgrade -q google-genai a2a-sdk==0.3.0 python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [7]:
# Targeted workaround for google-adk==1.9.0 compatibility with a2a-sdk==0.3.0
# This cell shall be removed when google-adk releases the version next to >1.9.0
# (after https://github.com/google/adk-python/pull/2297)


import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver


class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [8]:
import asyncio
import logging
import os
import sys
import threading
import time

from typing import Any

import httpx
import nest_asyncio
import uvicorn

from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search


In [9]:
import asyncio
import json
import sqlite3

from mcp import types as mcp_types
from mcp.server.lowlevel import Server
from mcp.server.models import InitializationOptions
import mcp.server.stdio

from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.mcp_tool.conversion_utils import adk_to_mcp_tool_type

In [10]:
# Set Google Cloud Configuration
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"      # use Generative AI API, not Vertex endpoint
os.environ["GOOGLE_CLOUD_PROJECT"] = "multiagenta2a"   # <-- your project id here, NO brackets
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"    # same as in the notes

load_dotenv()
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

print('Environment variables configured:')
print(f'GOOGLE_GENAI_USE_VERTEXAI: {os.environ["GOOGLE_GENAI_USE_VERTEXAI"]}')
print(f'GOOGLE_CLOUD_PROJECT: {os.environ["GOOGLE_CLOUD_PROJECT"]}')
print(f'GOOGLE_CLOUD_LOCATION: {os.environ["GOOGLE_CLOUD_LOCATION"]}')

Environment variables configured:
GOOGLE_GENAI_USE_VERTEXAI: FALSE
GOOGLE_CLOUD_PROJECT: multiagenta2a
GOOGLE_CLOUD_LOCATION: us-central1


In [11]:
# Authenticate your notebook environment (Colab only)
if 'google.colab' in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=os.environ['GOOGLE_CLOUD_PROJECT'])

## MCP Class Setup

In [12]:
!pip install fastmcp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.6/385.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: referencing
    Found existing installation: referencing 0.37.0
    Uninstalling referencing-0.37.0:
      Successfully uninstalled referencing-0.37.0


In [13]:
db = DatabaseSetup("support.db")
db.connect()

Connected to database: support.db


In [14]:
import sqlite3
import threading
import asyncio
from fastmcp import FastMCP

# Create FastMCP app
mcp_app = FastMCP("SupportMCP")

# Utility to get DB connection
def get_conn():
    conn = sqlite3.connect("support.db", check_same_thread=False)
    conn.row_factory = sqlite3.Row
    return conn

# -----------------------------
# 1. get_customer
# -----------------------------
@mcp_app.tool()
def get_customer(customer_id: int):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT * FROM customers WHERE id=?", (customer_id,))
    row = cur.fetchone()
    conn.close()
    return {"customer": dict(row) if row else None}

# -----------------------------
# 2. list_customers
# -----------------------------
@mcp_app.tool()
def list_customers(status: str = "active", limit: int = 50):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT * FROM customers WHERE status=? LIMIT ?", (status, limit))
    rows = cur.fetchall()
    conn.close()
    return {"customers": [dict(r) for r in rows]}

# -----------------------------
# 3. update_customer
# -----------------------------
@mcp_app.tool()
def update_customer(customer_id: int, data: dict):
    conn = get_conn()
    cur = conn.cursor()
    for field, val in data.items():
        cur.execute(f"UPDATE customers SET {field}=? WHERE id=?", (val, customer_id))
    conn.commit()
    conn.close()
    return {"updated": True}

# -----------------------------
# 4. create_ticket
# -----------------------------
@mcp_app.tool()
def create_ticket(customer_id: int, issue: str, priority: str = "medium"):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("""
        INSERT INTO tickets (customer_id, issue, status, priority)
        VALUES (?, ?, 'open', ?)
    """, (customer_id, issue, priority))
    conn.commit()
    conn.close()
    return {"ticket_created": True}

# -----------------------------
# 5. get_customer_history
# -----------------------------
@mcp_app.tool()
def get_customer_history(customer_id: int):
    conn = get_conn()
    cur = conn.cursor()
    cur.execute("SELECT * FROM tickets WHERE customer_id=?", (customer_id,))
    rows = cur.fetchall()
    conn.close()
    return {"history": [dict(r) for r in rows]}

# Run the FastMCP server on port 8001
def start_fast_mcp():
    asyncio.run(
        mcp_app.run_async(
            transport="http",
            host="127.0.0.1",
            port=8001
        )
    )

threading.Thread(target=start_fast_mcp, daemon=True).start()

print("FastMCP server running at http://127.0.0.1:8001/mcp")

FastMCP server running at http://127.0.0.1:8001/mcp


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [15]:
from google.adk.tools.mcp_tool import McpToolset, StreamableHTTPConnectionParams

connection_params = StreamableHTTPConnectionParams(
    url="http://127.0.0.1:8001/mcp"
)

# Instance used ONLY for notebook tests, never passed to agents
test_mcp_tools = McpToolset(connection_params=connection_params)

# Separate fresh instance passed into ADK agents
agent_mcp_tools = McpToolset(connection_params=connection_params)

TESTING MCPTOOLSET

In [16]:
from google.adk.tools.mcp_tool import MCPTool
import inspect

print("=== MCPTool methods ===")
for name, func in inspect.getmembers(MCPTool, inspect.isfunction):
    print("-", name)

print("\n=== MCPTool attributes ===")
print([a for a in dir(MCPTool) if not a.startswith("_")])

=== MCPTool methods ===
- __init__
- _get_declaration
- _get_headers
- _invoke_callable
- _run_async_impl
- process_llm_request
- run_async

=== MCPTool attributes ===
['custom_metadata', 'from_config', 'is_long_running', 'process_llm_request', 'raw_mcp_tool', 'run_async']


[12/04/25 00:58:08] INFO     Starting MCP server 'SupportMCP' with transport 'http' on               ]8;id=49434;file:///usr/local/lib/python3.12/dist-packages/fastmcp/server/server.py\server.py]8;;\:]8;id=684588;file:///usr/local/lib/python3.12/dist-packages/fastmcp/server/server.py#2058\2058]8;;\
                             http://127.0.0.1:8001/mcp                                                             

In [17]:
import asyncio

async def test_mcp_connection():
    print("Testing MCPToolset → FastMCP connection…")

    # Load tools from MCP server
    tools = await test_mcp_tools.get_tools()

    print(f"✔ Loaded {len(tools)} tools from MCP server")
    for t in tools:
        print(" -", t.name)

    # Try calling one tool
    for t in tools:
        if t.name == "list_customers":
            print("\nCalling list_customers(status='active', limit=3)…")
            result = await t.run_async(
                args={"status": "active", "limit": 3},
                tool_context=None
            )
            print("Result:", result)
            break

await test_mcp_connection()


Testing MCPToolset → FastMCP connection…


INFO:     Started server process [1432]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:37742 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:37748 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:37760 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:37776 - "POST /mcp HTTP/1.1" 200 OK
✔ Loaded 5 tools from MCP server
 - get_customer
 - list_customers
 - update_customer
 - create_ticket
 - get_customer_history

Calling list_customers(status='active', limit=3)…
INFO:     127.0.0.1:37790 - "POST /mcp HTTP/1.1" 200 OK
Result: {'content': [{'type': 'text', 'text': '{"customers":[{"id":1,"name":"John Doe","email":"john.doe@example.com","phone":"+1-555-0101","status":"active","created_at":"2025-12-04 00:57:17","updated_at":"2025-12-04 00:57:17"},{"id":2,"name":"Jane Smith","email":"jane.smith@example.com","phone":"+1-555-0102","status":"active","created_at":"2025-12-04 00:57:17","updated_at":"2025-12-04 00:57:17"},{"id":4,"name":"Alice Williams","email":"alice.w@techcorp.com","phone":"+1-555-0104","status":"active","created_at":"2025-12-04 00:57:17

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_toolset.py:196: DeprecationWarning: MCPTool class is deprecated, use `McpTool` instead.
  mcp_tool = MCPTool(
/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:103: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects

## Agents and Cards

## Customer Data

In [18]:
from google.adk.agents import Agent
from a2a.types import AgentCard, AgentCapabilities, AgentSkill, TransportProtocol

# ---------------------------
# Customer Data Agent (ADK)
# ---------------------------
customer_data_agent = Agent(
    model="gemini-2.5-pro",
    name="customer_data_agent",
    instruction="""
You are the Data Agent.

Your job: execute structured database tasks requested by the Router.
You NEVER generate prose.
You NEVER interpret user intent.
You ONLY fill data_result with structured results from MCP tool calls.

You must NEVER modify any fields except `data_result`.
You must NEVER delete fields.
You must NEVER return plain strings.
You must ALWAYS return the full JSON packet.

=====================================================================
INPUT JSON (ALWAYS PROVIDED TO YOU)
=====================================================================
{
  "scenario": "...",
  "input_message": "...",
  "task": "...",
  "params": {...},
  "data_result": null OR {...},
  "support_response": null,
  "final_answer": null
}

=====================================================================
AVAILABLE MCP TOOLS — YOU CAN CALL ONLY THESE
=====================================================================
1. get_customer(customer_id:int)
      → {"customer": {...}} or {"customer": None}

2. list_customers(status:str="active", limit:int=50)
      → {"customers": [ {...}, {...} ]}

3. update_customer(customer_id:int, data:dict)
      → {"updated": True}

4. create_ticket(customer_id:int, issue:str, priority:str)
      → {"ticket_created": True}

5. get_customer_history(customer_id:int)
      → {"history": [ {...ticket...}, {...ticket...} ]}

You may call tools multiple times inside one task (e.g., looping through IDs).

=====================================================================
TASK DEFINITIONS — YOU MUST FOLLOW THESE EXACT RULES
=====================================================================

---------------------------------------------------------------------
1. TASK: "fetch_customer_info"
---------------------------------------------------------------------
REQUIRES:
   params.customer_id must exist.

YOU MUST:
   - Call the MCP tool get_customer(customer_id)
     using the exact ID supplied.
   - Never guess customer data.
   - Never fabricate fields.

EXAMPLE INTERNAL THINKING (NOT OUTPUT):
   tool.get_customer({"customer_id": 5})
   → returns {"customer": {...}}

OUTPUT:
   You must produce:
   {
     "scenario": "...",
     "input_message": "...",
     "task": "fetch_customer_info",
     "params": {...},
     "data_result": {"customer": {... or None}},
     "support_response": null,
     "final_answer": null
   }

If customer_id missing:
   data_result = {"error": "missing_parameters"}


---------------------------------------------------------------------
2. TASK: "fetch_active_customers"
---------------------------------------------------------------------
YOU MUST:
   - Call list_customers(status="active")
   - Extract the "customers" returned by the tool
   - Place them under:
         data_result = {"active_customers": [...]}

OUTPUT FORMAT (full JSON preserved):
{
  "scenario": "...",
  "input_message": "...",
  "task": "fetch_active_customers",
  "params": {...},
  "data_result": {"active_customers": [...]},
  "support_response": null,
  "final_answer": null
}


---------------------------------------------------------------------
3. TASK: "fetch_ticket_history"
---------------------------------------------------------------------
REQUIRES:
   params.customer_ids must exist AND be a list of ints.

YOU MUST:
   - For each id in params.customer_ids:
         call get_customer_history(id)
   - Inspect tool result: history = [{"status": "...", ...}, ...]
   - If ANY ticket for that customer has status == "open":
         include this ID in open_ticket_ids.

YOU MUST NOT:
   - Overwrite or erase previously stored fields in data_result.
   - Lose any previously stored active_customers.

You MUST:
   - Append or insert the new field:
         "open_ticket_customers": [list_of_ids]

Final data_result must include ALL accumulated fields.

EXAMPLE FINAL data_result:
{
  "active_customers": [...],
  "open_ticket_customers": [1, 4, 5, 10]
}

OUTPUT FORMAT:
{
  "scenario": "...",
  "input_message": "...",
  "task": "fetch_ticket_history",
  "params": {...},
  "data_result": {"active_customers": [...], "open_ticket_customers": [...]},
  "support_response": null,
  "final_answer": null
}

If params.customer_ids missing:
   data_result = {"error": "missing_parameters"}


=====================================================================
ABSOLUTE OUTPUT RULES
=====================================================================
1. ALWAYS return the FULL JSON packet you were given.
2. ONLY replace the contents of `data_result`.
3. NEVER modify scenario, input_message, task, params, support_response, final_answer.
4. NEVER output strings, prose, or commentary.
5. NEVER invent data. You must ALWAYS call the appropriate MCP tool.
6. NEVER remove fields from data_result during multi-step flows.
7. ALWAYS merge new fields into data_result for multi-step flows.

=====================================================================
END OF DATA AGENT RULES
=====================================================================


""",
     tools=[agent_mcp_tools],
)


print("✓ Customer Data Agent created and wired to MCPToolset.")

# ---------------------------
# Customer Data AgentCard (A2A metadata)
# ---------------------------
customer_data_agent_card = AgentCard(
    name="Customer Data Agent",
    url="http://localhost:10030",
    description="Provides MCP-backed access to customer and ticket data.",
    version="1.0",
    capabilities=AgentCapabilities(streaming=False),
    default_input_modes=["text/plain"],
    default_output_modes=["application/json"],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id="db_access",
            name="Database Access",
            description="Reads and updates customers and tickets via MCP.",
            tags=["mcp", "database", "customers", "tickets"],
            examples=[
                "Get customer information for ID 5",
                "List active customers",
                "Show ticket history for customer 3",
            ],
        )
    ],
)

print("✓ Customer Data AgentCard created.")


✓ Customer Data Agent created and wired to MCPToolset.
✓ Customer Data AgentCard created.


## SUPPORT

In [19]:
# ============================================================
# PHASE 3 — SUPPORT AGENT (LLM reasoning + negotiation)
# ============================================================

from google.adk.agents import Agent
from a2a.types import AgentCard, AgentCapabilities, AgentSkill, TransportProtocol


support_agent = Agent(
    model="gemini-2.5-pro",
    name="support_agent",
    instruction="""
You are the Support Agent.

Your job:
- Interpret the user's original request (input_message)
- Decide whether you can answer using available data_result
- OR decide what additional data you need from the Data Agent

You NEVER call the Data Agent yourself — only request data via:
{
  "support_response": {
      "type": "needs_data",
      "need": "<task_name>",
      "params": {...}
  },
  "final_answer": null
}

You NEVER modify ANY other JSON fields. Only fill support_response OR final_answer.

=====================================================================
INPUT JSON
=====================================================================
{
  "scenario": "...",
  "input_message": "...",
  "task": "...",
  "params": {...},
  "data_result": null or {...},
  "support_response": null,
  "final_answer": null
}

=====================================================================
SCENARIO LOGIC
=====================================================================

======================
SCENARIO 1 — SIMPLE
======================
User asks for information of a single customer by ID (router gives the ID).
Steps:

- If data_result is null:
    Request:
    support_response = {
       "type": "needs_data",
       "need": "fetch_customer_info",
       "params": {"customer_id": X}
    }

- If data_result has a full customer record:
    You may freely generate a final_answer that best matches the user's input_message.
    Your final_answer must:
        • directly answer the user's query,
        • use the retrieved customer fields (name, status, email, phone if needed),
        • stay helpful and concise,
        • NOT follow a fixed template.

Examples:
- For queries like "Get customer information for ID 5":
      "Customer Charlie Brown is active. Their email is charlie.brown@email.com."

- For queries like "I need help with my account, customer ID 5":
      "I've found customer Charlie Brown, whose account is active. How can I help with the account issue you mentioned?"

You have freedom to phrase the answer naturally as long as it is accurate.

==============================
SCENARIO 2 — NEGOTIATION_ESCALATION
==============================

IMPORTANT:
Only in negotiation/escalation do you ask for missing customer ID.

### 1. If customer_id not found AND scenario=negotiation_escalation:
Return urgent or normal ID-request.

Urgent keywords (case-insensitive):
"charged twice", "refund", "immediately", "urgent", "asap", "right away", "emergency"

If urgent:
final_answer =
"I'm really sorry to hear about this urgent issue. I want to resolve it as quickly as possible.
Could you please provide your customer ID so I can look into this immediately?"

Else:
final_answer =
"I'm happy to help! Before I can continue, could you please provide your customer ID?"

### 2. If customer_id exists but no data_result:
Request:
{
  "type": "needs_data",
  "need": "fetch_customer_info",
  "params": {"customer_id": X}
}

### 3. If data_result exists:
Produce helpful final_answer incorporating urgency if detected.

==============================
SCENARIO 3 — MULTI_STEP
==============================
User asks about groups, not individuals.

Examples:
"Show me all active customers who have open tickets"

This scenario never asks for customer_id.

Process:

Step 1:
If no "active_customers" in data_result:
    Request:
    {
       "type":"needs_data",
       "need":"fetch_active_customers"
    }

Step 2:
If active_customers exists but no open_ticket_customers:
    Extract IDs = [c["id"] for c in active_customers]
    Request:
    {
      "type":"needs_data",
      "need":"fetch_ticket_history",
      "params":{"customer_ids": IDs}
    }

Step 3:
If both active_customers AND open_ticket_customers exist:
    Compose final_answer:
    - List only customers whose IDs are in open_ticket_customers
    - Provide a clean summary

=====================================================================
OUTPUT RULES
=====================================================================

You fill ONLY ONE field:
- support_response  OR
- final_answer

Never fill both.
Never modify other fields.

Output JSON only.
""",
    tools=[]  # IMPORTANT: Support Agent has NO MCP tools
)


print("✓ Support Agent created.")


# ---------------------------
# Support AgentCard (A2A metadata)
# ---------------------------
support_agent_card = AgentCard(
    name="Support Agent",
    url="http://localhost:10031",
    description="Handles negotiation, clarification, and escalation logic for customer support.",
    version="1.0",
    capabilities=AgentCapabilities(streaming=False),
    default_input_modes=["text/plain"],
    default_output_modes=["application/json"],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id="negotiation",
            name="Negotiation and Clarification",
            description="Handles negotiation scenarios (Scenario 2), including requests for billing context or clarification.",
            tags=["support", "negotiation", "billing", "customer_service"],
            examples=[
                "The user wants to cancel but also has billing issues.",
                "The user is confused and needs clarification.",
                "Ask the user politely for more details.",
            ],
        ),
        AgentSkill(
            id="escalation",
            name="Escalation Handling",
            description="Handles severe complaints and escalates when appropriate.",
            tags=["escalation", "complaint", "refund", "supervisor"],
            examples=[
                "The user demands a refund.",
                "The user requests a supervisor.",
                "The user expresses extreme frustration.",
            ],
        )
    ],
)

print("✓ Support Agent Card created.")


✓ Support Agent created.
✓ Support Agent Card created.


## ROUTER

In [20]:
# ============================================================
# PHASE 4 — ROUTER AGENT
# ============================================================

from google.adk.agents import Agent
from a2a.types import AgentCard, AgentCapabilities, AgentSkill, TransportProtocol
import re
import json


router_agent = Agent(
    model="gemini-2.5-pro",
    name="router_agent",
    instruction="""
You are the Router Agent.

Your job: **coordinate all other agents** (SUPPORT and DATA) by:
1. Identifying the scenario type (simple, negotiation_escalation, multi_step)
2. Delegating to SUPPORT when interpretation is needed
3. Delegating to DATA when structured data is needed
4. NEVER answering the user yourself
5. NEVER deleting or altering any JSON fields other than `task` and `params`

=====================================================================
INPUT JSON (from user or another agent)
=====================================================================
{
  "scenario": null or "simple" or "negotiation_escalation" or "multi_step",
  "input_message": "...original user query...",
  "task": null,
  "params": {},
  "data_result": null,
  "support_response": null,
  "final_answer": null
}

You must ALWAYS return the **entire JSON**, preserving all fields.

=====================================================================
SCENARIO DETECTION RULES (FIRST PASS ONLY)
=====================================================================

If scenario is null (first run):

- SIMPLE:
    Triggered when user asks:
      - "Get customer information for ID X"
      - "What is the info for customer X?"

- NEGOTIATION_ESCALATION:
    Triggered for:
      - refund requests
      - cancellation + billing
      - account upgrade help
      - customer-specific issues requiring ID

    Examples:
      "I'm customer 5 and need help upgrading my account"
      "I've been charged twice, please refund immediately!"
      "I want to cancel my subscription but I'm having billing issues"

- MULTI_STEP:
    Triggered for queries asking about:
      - sets of customers
      - aggregate conditions
      - multi-stage data retrieval

    Examples:
      "Show me all active customers who have open tickets"
      "What's the status of all high-priority tickets for premium customers?"

Router must set scenario accordingly.

=====================================================================
ROUTER LOGIC
=====================================================================

### 1. If final_answer is already filled:
Return JSON unchanged.

### 2. If support_response exists:
Router must translate support_response into a DATA AGENT request.

Support will produce:
{
  "type": "needs_data",
  "need": "<data_task_name>",
  "params": {...optional...}
}

Router must translate it into:

task = support_response.need
params = support_response.params or {}

Examples:
- need: "fetch_customer_info"  → task="fetch_customer_info"
- need: "fetch_active_customers" → task="fetch_active_customers"
- need: "fetch_ticket_history" → task="fetch_ticket_history"

Router NEVER chooses data tasks itself.

### 3. After receiving DATA_RESULT from Data Agent:
Do NOT modify data_result.
Instead:
- Reset support_response to null
- Set task to:
  "Check whether you now have enough data to answer the user's input_message.
   If not, say what you still need."

Then send back to SUPPORT unchanged.

=====================================================================
OUTPUT FORMAT
=====================================================================

You must always return JSON in the following format:

{
  "scenario": "...",
  "input_message": "...same...",
  "task": "...possibly updated...",
  "params": { ...possibly updated... },
  "data_result": ...preserved or updated...,
  "support_response": null or { ... },
  "final_answer": null or "...string..."
}

Return ONLY JSON — no explanations.
"""
)


print("✓ Router Agent created.")


# ---------------------------
# Router AgentCard (A2A metadata)
# ---------------------------
router_agent_card = AgentCard(
    name="Router Agent",
    description="Routes user queries to Data or Support agents based on scenario.",
    url="http://localhost:10032",
    version="1.0",
    capabilities=AgentCapabilities(streaming=False),
    default_input_modes=["text/plain"],
    default_output_modes=["application/json"],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id="intent_detection",
            name="Intent Detection",
            description="Classifies queries for routing to Data or Support agents.",
            tags=["routing", "intent", "classification"],
            examples=[
                "Show me customer 3",
                "Find all active users",
                "I want to cancel my subscription but I also have billing issues",
                "This is ridiculous! I want a refund now!"
            ]
        )
    ],
)


print("✓ Router Agent Card created.")


✓ Router Agent created.
✓ Router Agent Card created.


In [21]:
print("MCP server restarting...")

# Use ONLY the test toolset for notebook checks
tools = await test_mcp_tools.get_tools()
print("✔ Tools loaded from MCP (notebook test):")
tools

MCP server restarting...
INFO:     127.0.0.1:37802 - "POST /mcp HTTP/1.1" 200 OK
✔ Tools loaded from MCP (notebook test):


/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_toolset.py:196: DeprecationWarning: MCPTool class is deprecated, use `McpTool` instead.
  mcp_tool = MCPTool(
/usr/local/lib/python3.12/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:103: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## A2A

In [22]:
# ============================================================
# PHASE 5 — A2A SERVERS FOR ALL 3 AGENTS
# ============================================================

from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
import uvicorn
import nest_asyncio
nest_asyncio.apply()

def create_agent_a2a_server(agent, agent_card):
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )
    executor = A2aAgentExecutor(
        runner=runner,
        config=A2aAgentExecutorConfig(),
    )
    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )
    return A2AStarletteApplication(
        agent_card=agent_card,
        http_handler=request_handler,
    )

async def run_customer_data_server():
    app = create_agent_a2a_server(customer_data_agent, customer_data_agent_card)
    config = uvicorn.Config(app.build(), host="127.0.0.1", port=10030, log_level="warning", loop="none")
    server = uvicorn.Server(config)
    await server.serve()

async def run_support_agent_server():
    app = create_agent_a2a_server(support_agent, support_agent_card)
    config = uvicorn.Config(app.build(), host="127.0.0.1", port=10031, log_level="warning", loop="none")
    server = uvicorn.Server(config)
    await server.serve()

async def run_router_agent_server():
    app = create_agent_a2a_server(router_agent, router_agent_card)
    config = uvicorn.Config(app.build(), host="127.0.0.1", port=10032, log_level="warning", loop="none")
    server = uvicorn.Server(config)
    await server.serve()

async def start_all_servers():
    print("Starting all 3 agents...")
    tasks = [
        asyncio.create_task(run_customer_data_server()),
        asyncio.create_task(run_support_agent_server()),
        asyncio.create_task(run_router_agent_server()),
    ]
    await asyncio.sleep(6)
    print("✅ All agent servers started:")
    await asyncio.gather(*tasks)

def run_servers_in_background():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())

server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

import time
time.sleep(3)
print("All three A2A servers running.")


/tmp/ipython-input-172568068.py:30: UserWarning: [EXPERIMENTAL] A2aAgentExecutorConfig: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  config=A2aAgentExecutorConfig(),
/tmp/ipython-input-172568068.py:28: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  executor = A2aAgentExecutor(


Starting all 3 agents...


/usr/local/lib/python3.12/dist-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol


All three A2A servers running.


## A2ASimple Client

In [23]:
# ============================================================
# PHASE 6 — SIMPLE A2A CLIENT + CLEAN OUTPUT EXTRACTION
# ============================================================

import httpx
import json
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.types import AgentCard, TransportProtocol


# -----------------------------------------------------------
# Helper: Extract structuredContent from ADK task output
# -----------------------------------------------------------
def extract_structured(task):
    """
    Universal extractor for Router, DataAgent, SupportAgent
    Handles:
    - .root.data (DataAgent structured)
    - .root.text containing JSON
    - plain text fallback
    """
    try:
        art = task.artifacts[0]
        part = art.parts[0]
        root = part.root

        # ----------------------------------------------------
        # 1) Case: DataAgent structuredContent
        # ----------------------------------------------------
        if hasattr(root, "data") and isinstance(root.data, dict):
            data = root.data
            if (
                "response" in data
                and isinstance(data["response"], dict)
                and "structuredContent" in data["response"]
            ):
                return data["response"]["structuredContent"]

            # Router sometimes returns dict in .data
            return data

        # ----------------------------------------------------
        # 2) Case: Router/Support: text containing JSON
        # ----------------------------------------------------
        if hasattr(root, "text") and isinstance(root.text, str):
            text = root.text.strip()

            # Remove ```json fences
            if text.startswith("```"):
                text = text.strip("`")
                if text.startswith("json"):
                    text = text[4:].strip()

            # Try JSON decode
            try:
                return json.loads(text)
            except Exception:
                return text  # plain text fallback

    except Exception:
        pass

    return None



# -----------------------------------------------------------
# A2ASimpleClient — fully corrected version
# -----------------------------------------------------------
class A2ASimpleClient:
    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache = {}
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message, clean=True):
        """
        agent_url : http://127.0.0.1:PORT
        message   : str OR dict
        clean     : if True, returns structured JSON instead of ADK dump
        """

        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:

            # ---------------------------
            # Load AgentCard (cached)
            # ---------------------------
            if agent_url not in self._agent_info_cache:
                card_resp = await httpx_client.get(
                    f"{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}"
                )
                self._agent_info_cache[agent_url] = card_resp.json()

            agent_card = AgentCard(**self._agent_info_cache[agent_url])

            # ---------------------------
            # Create A2A client
            # ---------------------------
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            client = ClientFactory(config).create(agent_card)

            # Convert dict → JSON string
            if isinstance(message, dict):
                message = json.dumps(message)

            msg = create_text_message_object(content=message)

            # ---------------------------
            # Send message
            # ---------------------------
            responses = []
            async for response in client.send_message(msg):
                responses.append(response)

            if not responses or not isinstance(responses[0], tuple):
                return "No response received"

            task = responses[0][0]

            # ---------------------------
            # Clean extraction (preferred)
            # ---------------------------
            if clean:
                cleaned = extract_structured(task)
                return cleaned if cleaned is not None else str(task)

            # ---------------------------
            # Raw fallback
            # ---------------------------
            return str(task)


# GLOBAL CLIENT INSTANCE
a2a_client = A2ASimpleClient()


In [24]:
import logging
import warnings

# ======================================================================
#  BASE LOGGING CONFIG — suppress almost everything
# ======================================================================
logging.basicConfig(
    level=logging.CRITICAL,        # CRITICAL = only show catastrophic errors
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
)

# ======================================================================
#  SILENCE all noisy subsystems (ADK, aiohttp, httpx, jupyter, asyncio)
# ======================================================================
noisy_modules = [
    "google.adk", "google_adk",
    "google.genai", "google_genai",
    "aiohttp", "httpx",
    "anyio", "urllib3",
    "jupyter_client", "asyncio",
    "mcp", "MCPTool"
]

for module in noisy_modules:
    logging.getLogger(module).setLevel(logging.CRITICAL)

# ======================================================================
#  SILENCE Python warnings (Deprecation, Runtime, FutureWarning, etc.)
# ======================================================================
warnings.filterwarnings("ignore")

# EXTRA: reduce asyncio debug noise
logging.getLogger("asyncio").setLevel(logging.CRITICAL)


## TEST SCENARIOS

In [25]:
ROUTER_URL = "http://127.0.0.1:10032"
DATA_URL   = "http://127.0.0.1:10030"
SUPPORT_URL = "http://127.0.0.1:10031"

In [26]:
import logging
import warnings

# ======================================================================
#  BASE LOGGING CONFIG — suppress almost everything
# ======================================================================
logging.basicConfig(
    level=logging.CRITICAL,        # CRITICAL = only show catastrophic errors
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
)

# ======================================================================
#  SILENCE all noisy subsystems (ADK, aiohttp, httpx, jupyter, asyncio)
# ======================================================================
noisy_modules = [
    "google.adk", "google_adk",
    "google.genai", "google_genai",
    "aiohttp", "httpx",
    "anyio", "urllib3",
    "jupyter_client", "asyncio",
    "mcp", "MCPTool"
]

for module in noisy_modules:
    logging.getLogger(module).setLevel(logging.CRITICAL)

# ======================================================================
#  SILENCE Python warnings (Deprecation, Runtime, FutureWarning, etc.)
# ======================================================================
warnings.filterwarnings("ignore")

# EXTRA: reduce asyncio debug noise
logging.getLogger("asyncio").setLevel(logging.CRITICAL)


## Manual Sequence Test: Simple Scenario

In [27]:
resp_router = await a2a_client.create_task(
    ROUTER_URL,
    "Get customer information for ID 5"
)
resp_router

✅ All agent servers started:


{'scenario': 'simple',
 'input_message': 'Get customer information for ID 5',
 'task': "Interpret the user's `input_message` and determine the next action. If data is needed, specify the `need` and any `params`.",
 'params': {},
 'data_result': None,
 'support_response': None,
 'final_answer': None}

In [28]:
resp_support = await a2a_client.create_task(
    SUPPORT_URL,
    resp_router
)
resp_support

{'scenario': 'simple',
 'input_message': 'Get customer information for ID 5',
 'task': "Interpret the user's `input_message` and determine the next action. If data is needed, specify the `need` and any `params`.",
 'params': {},
 'data_result': None,
 'support_response': {'type': 'needs_data',
  'need': 'fetch_customer_info',
  'params': {'customer_id': 5}},
 'final_answer': None}

In [29]:
resp_router = await a2a_client.create_task(
    ROUTER_URL,
    resp_support
)
resp_router


{'scenario': 'simple',
 'input_message': 'Get customer information for ID 5',
 'task': 'fetch_customer_info',
 'params': {'customer_id': 5},
 'data_result': None,
 'support_response': {'type': 'needs_data',
  'need': 'fetch_customer_info',
  'params': {'customer_id': 5}},
 'final_answer': None}

In [30]:
resp_data = await a2a_client.create_task(
    DATA_URL,
    resp_router
)
resp_data


{'scenario': 'simple',
 'input_message': 'Get customer information for ID 5',
 'task': 'fetch_customer_info',
 'params': {'customer_id': 5},
 'data_result': {'customer': {'id': 5,
   'name': 'Charlie Brown',
   'email': 'charlie.brown@email.com',
   'phone': '+1-555-0105',
   'status': 'active',
   'created_at': '2025-12-04 00:57:17',
   'updated_at': '2025-12-04 00:57:17'}},
 'support_response': {'type': 'needs_data',
  'need': 'fetch_customer_info',
  'params': {'customer_id': 5}},
 'final_answer': None}

In [31]:
resp_router = await a2a_client.create_task(
    ROUTER_URL,
    resp_data
)
resp_router

{'scenario': 'simple',
 'input_message': 'Get customer information for ID 5',
 'task': "Check whether you now have enough data to answer the user's input_message. If not, say what you still need.",
 'params': {},
 'data_result': {'customer': {'id': 5,
   'name': 'Charlie Brown',
   'email': 'charlie.brown@email.com',
   'phone': '+1-555-0105',
   'status': 'active',
   'created_at': '2025-12-04 00:57:17',
   'updated_at': '2025-12-04 00:57:17'}},
 'support_response': None,
 'final_answer': None}

In [32]:

resp_support = await a2a_client.create_task(
    SUPPORT_URL,
    resp_router
)
resp_support


{'scenario': 'simple',
 'input_message': 'Get customer information for ID 5',
 'task': "Check whether you now have enough data to answer the user's input_message. If not, say what you still need.",
 'params': {},
 'data_result': {'customer': {'id': 5,
   'name': 'Charlie Brown',
   'email': 'charlie.brown@email.com',
   'phone': '+1-555-0105',
   'status': 'active',
   'created_at': '2025-12-04 00:57:17',
   'updated_at': '2025-12-04 00:57:17'}},
 'support_response': None,
 'final_answer': 'Customer Charlie Brown is active. Their email is charlie.brown@email.com.'}

In [33]:
resp_router = await a2a_client.create_task(
    ROUTER_URL,
    resp_support
)
resp_router


{'scenario': 'simple',
 'input_message': 'Get customer information for ID 5',
 'task': "Check whether you now have enough data to answer the user's input_message. If not, say what you still need.",
 'params': {},
 'data_result': {'customer': {'id': 5,
   'name': 'Charlie Brown',
   'email': 'charlie.brown@email.com',
   'phone': '+1-555-0105',
   'status': 'active',
   'created_at': '2025-12-04 00:57:17',
   'updated_at': '2025-12-04 00:57:17'}},
 'support_response': None,
 'final_answer': 'Customer Charlie Brown is active. Their email is charlie.brown@email.com.'}

In [34]:
final_answer = resp_router.get("final_answer")
final_answer

'Customer Charlie Brown is active. Their email is charlie.brown@email.com.'

## NEGOTIATION

In [35]:
msg1 = "I'm customer 5 and need help upgrading my account"

router_1 = await a2a_client.create_task(
    ROUTER_URL,
    msg1
)
router_1

{'scenario': 'negotiation_escalation',
 'input_message': "I'm customer 5 and need help upgrading my account",
 'task': "Interpret the user's request and identify the next step.",
 'params': {},
 'data_result': None,
 'support_response': None,
 'final_answer': None}

In [36]:
support_1 = await a2a_client.create_task(
    SUPPORT_URL,
    router_1
)
support_1


{'scenario': 'negotiation_escalation',
 'input_message': "I'm customer 5 and need help upgrading my account",
 'task': "Interpret the user's request and identify the next step.",
 'params': {},
 'data_result': None,
 'support_response': {'type': 'needs_data',
  'need': 'fetch_customer_info',
  'params': {'customer_id': 5}},
 'final_answer': None}

In [37]:
router_2 = await a2a_client.create_task(
    ROUTER_URL,
    support_1
)
router_2


{'scenario': 'negotiation_escalation',
 'input_message': "I'm customer 5 and need help upgrading my account",
 'task': 'fetch_customer_info',
 'params': {'customer_id': 5},
 'data_result': None,
 'support_response': {'type': 'needs_data',
  'need': 'fetch_customer_info',
  'params': {'customer_id': 5}},
 'final_answer': None}

In [38]:
data_1 = await a2a_client.create_task(
    DATA_URL,
    router_2
)
data_1


{'scenario': 'negotiation_escalation',
 'input_message': "I'm customer 5 and need help upgrading my account",
 'task': 'fetch_customer_info',
 'params': {'customer_id': 5},
 'data_result': {'customer': {'id': 5,
   'name': 'Charlie Brown',
   'email': 'charlie.brown@email.com',
   'phone': '+1-555-0105',
   'status': 'active',
   'created_at': '2025-12-03 23:40:38',
   'updated_at': '2025-12-03 23:40:38'}},
 'support_response': {'type': 'needs_data',
  'need': 'fetch_customer_info',
  'params': {'customer_id': 5}},
 'final_answer': None}

In [39]:
router_3 = await a2a_client.create_task(
    ROUTER_URL,
    data_1
)
router_3


{'scenario': 'negotiation_escalation',
 'input_message': "I'm customer 5 and need help upgrading my account",
 'task': "Check whether you now have enough data to answer the user's input_message. If not, say what you still need.",
 'params': {},
 'data_result': {'customer': {'id': 5,
   'name': 'Charlie Brown',
   'email': 'charlie.brown@email.com',
   'phone': '+1-555-0105',
   'status': 'active',
   'created_at': '2025-12-03 23:40:38',
   'updated_at': '2025-12-03 23:40:38'}},
 'support_response': None,
 'final_answer': None}

In [40]:
support_2 = await a2a_client.create_task(
    SUPPORT_URL,
    router_3
)
support_2


{'scenario': 'negotiation_escalation',
 'input_message': "I'm customer 5 and need help upgrading my account",
 'task': "Check whether you now have enough data to answer the user's input_message. If not, say what you still need.",
 'params': {},
 'data_result': {'customer': {'id': 5,
   'name': 'Charlie Brown',
   'email': 'charlie.brown@email.com',
   'phone': '+1-555-0105',
   'status': 'active',
   'created_at': '2025-12-03 23:40:38',
   'updated_at': '2025-12-03 23:40:38'}},
 'support_response': None,
 'final_answer': 'Hello Charlie Brown! I see your account is active and I can certainly help you with an upgrade. What kind of changes were you looking to make to your account?'}

In [41]:
router_final = await a2a_client.create_task(
    ROUTER_URL,
    support_2
)
router_final


{'scenario': 'negotiation_escalation',
 'input_message': "I'm customer 5 and need help upgrading my account",
 'task': "Check whether you now have enough data to answer the user's input_message. If not, say what you still need.",
 'params': {},
 'data_result': {'customer': {'id': 5,
   'name': 'Charlie Brown',
   'email': 'charlie.brown@email.com',
   'phone': '+1-555-0105',
   'status': 'active',
   'created_at': '2025-12-03 23:40:38',
   'updated_at': '2025-12-03 23:40:38'}},
 'support_response': None,
 'final_answer': 'Hello Charlie Brown! I see your account is active and I can certainly help you with an upgrade. What kind of changes were you looking to make to your account?'}

In [42]:
final_answer = router_final.get("final_answer")
final_answer

'Hello Charlie Brown! I see your account is active and I can certainly help you with an upgrade. What kind of changes were you looking to make to your account?'

## Emergency

In [43]:
msg1 = "I've been charged twice, please refund immediately!"

router_1 = await a2a_client.create_task(
    ROUTER_URL,
    msg1
)
router_1

{'scenario': 'negotiation_escalation',
 'input_message': "I've been charged twice, please refund immediately!",
 'task': "Please analyze the user's message and determine the next step.",
 'params': {},
 'data_result': None,
 'support_response': None,
 'final_answer': None}

In [44]:
support_1 = await a2a_client.create_task(
    SUPPORT_URL,
    router_1
)
support_1


{'scenario': 'negotiation_escalation',
 'input_message': "I've been charged twice, please refund immediately!",
 'task': "Please analyze the user's message and determine the next step.",
 'params': {},
 'data_result': None,
 'support_response': None,
 'final_answer': "I'm really sorry to hear about this urgent issue. I want to resolve it as quickly as possible. Could you please provide your customer ID so I can look into this immediately?"}

In [45]:
router_final = await a2a_client.create_task(
    ROUTER_URL,
    support_1
)
router_final

{'scenario': 'negotiation_escalation',
 'input_message': "I've been charged twice, please refund immediately!",
 'task': "Please analyze the user's message and determine the next step.",
 'params': {},
 'data_result': None,
 'support_response': None,
 'final_answer': "I'm really sorry to hear about this urgent issue. I want to resolve it as quickly as possible. Could you please provide your customer ID so I can look into this immediately?"}

##MultiStep Case

In [46]:
m_msg = "Show me all active customers who have open tickets"

m_router1 = await a2a_client.create_task(
    ROUTER_URL,
    m_msg
)
m_router1

{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': "Interpret the user's input_message and tell me what data is needed to answer it.",
 'params': {},
 'data_result': None,
 'support_response': None,
 'final_answer': None}

In [47]:
m_support1 = await a2a_client.create_task(
    SUPPORT_URL,
    m_router1
)
m_support1

{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': "Interpret the user's input_message and tell me what data is needed to answer it.",
 'params': {},
 'data_result': None,
 'support_response': {'type': 'needs_data',
  'need': 'fetch_active_customers',
  'params': {}},
 'final_answer': None}

In [48]:
m_router2 = await a2a_client.create_task(
    ROUTER_URL,
    m_support1
)
m_router2

{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': 'fetch_active_customers',
 'params': {},
 'data_result': None,
 'support_response': {'type': 'needs_data',
  'need': 'fetch_active_customers',
  'params': {}},
 'final_answer': None}

In [49]:
m_data1 = await a2a_client.create_task(
    DATA_URL,
    m_router2
)
m_data1

{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': 'fetch_active_customers',
 'params': {},
 'data_result': {'active_customers': [{'id': 1,
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'phone': '+1-555-0101',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 2,
    'name': 'Jane Smith',
    'email': 'jane.smith@example.com',
    'phone': '+1-555-0102',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 4,
    'name': 'Alice Williams',
    'email': 'alice.w@techcorp.com',
    'phone': '+1-555-0104',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 5,
    'name': 'Charlie Brown',
    'email': 'charlie.brown@email.com',
    'phone': '+1-555-0105',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    

In [50]:
m_router3 = await a2a_client.create_task(
    ROUTER_URL,
    m_data1
)
m_router3

{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': "Check whether you now have enough data to answer the user's input_message. \n   If not, say what you still need.",
 'params': {},
 'data_result': {'active_customers': [{'id': 1,
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'phone': '+1-555-0101',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 2,
    'name': 'Jane Smith',
    'email': 'jane.smith@example.com',
    'phone': '+1-555-0102',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 4,
    'name': 'Alice Williams',
    'email': 'alice.w@techcorp.com',
    'phone': '+1-555-0104',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 5,
    'name': 'Charlie Brown',
    'email': 'charlie.brown@email.com',
    'ph

In [51]:
m_support2 = await a2a_client.create_task(
    SUPPORT_URL,
    m_router3
)
m_support2

{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': "Check whether you now have enough data to answer the user's input_message. \n   If not, say what you still need.",
 'params': {},
 'data_result': {'active_customers': [{'id': 1,
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'phone': '+1-555-0101',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 2,
    'name': 'Jane Smith',
    'email': 'jane.smith@example.com',
    'phone': '+1-555-0102',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 4,
    'name': 'Alice Williams',
    'email': 'alice.w@techcorp.com',
    'phone': '+1-555-0104',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 5,
    'name': 'Charlie Brown',
    'email': 'charlie.brown@email.com',
    'ph

In [52]:
m_router4 = await a2a_client.create_task(
    ROUTER_URL,
    m_support2
)
m_router4

{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': 'fetch_ticket_history',
 'params': {'customer_ids': [1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 14, 15]},
 'data_result': {'active_customers': [{'id': 1,
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'phone': '+1-555-0101',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 2,
    'name': 'Jane Smith',
    'email': 'jane.smith@example.com',
    'phone': '+1-555-0102',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 4,
    'name': 'Alice Williams',
    'email': 'alice.w@techcorp.com',
    'phone': '+1-555-0104',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 5,
    'name': 'Charlie Brown',
    'email': 'charlie.brown@email.com',
    'phone': '+1-555-0105',
    'status': 

In [53]:
m_data2 = await a2a_client.create_task(
    DATA_URL,
    m_router4
)
m_data2

{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': 'fetch_ticket_history',
 'params': {'customer_ids': [1, 2, 4, 5, 6, 7, 9, 10, 11, 12, 14, 15]},
 'data_result': {'active_customers': [{'id': 1,
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'phone': '+1-555-0101',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 2,
    'name': 'Jane Smith',
    'email': 'jane.smith@example.com',
    'phone': '+1-555-0102',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 4,
    'name': 'Alice Williams',
    'email': 'alice.w@techcorp.com',
    'phone': '+1-555-0104',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 5,
    'name': 'Charlie Brown',
    'email': 'charlie.brown@email.com',
    'phone': '+1-555-0105',
    'status': 

In [54]:
m_router5 = await a2a_client.create_task(
    ROUTER_URL,
    m_data2
)
m_router5

{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': "Check whether you now have enough data to answer the user's input_message. If not, say what you still need.",
 'params': {},
 'data_result': {'active_customers': [{'id': 1,
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'phone': '+1-555-0101',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 2,
    'name': 'Jane Smith',
    'email': 'jane.smith@example.com',
    'phone': '+1-555-0102',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 4,
    'name': 'Alice Williams',
    'email': 'alice.w@techcorp.com',
    'phone': '+1-555-0104',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 5,
    'name': 'Charlie Brown',
    'email': 'charlie.brown@email.com',
    'phone':

In [55]:
m_support3 = await a2a_client.create_task(
    SUPPORT_URL,
    m_router5
)
m_support3


{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': "Check whether you now have enough data to answer the user's input_message. If not, say what you still need.",
 'params': {},
 'data_result': {'active_customers': [{'id': 1,
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'phone': '+1-555-0101',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 2,
    'name': 'Jane Smith',
    'email': 'jane.smith@example.com',
    'phone': '+1-555-0102',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 4,
    'name': 'Alice Williams',
    'email': 'alice.w@techcorp.com',
    'phone': '+1-555-0104',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 5,
    'name': 'Charlie Brown',
    'email': 'charlie.brown@email.com',
    'phone':

In [56]:
m_router_final = await a2a_client.create_task(
    ROUTER_URL,
    m_support3
)
m_router_final


{'scenario': 'multi_step',
 'input_message': 'Show me all active customers who have open tickets',
 'task': "Check whether you now have enough data to answer the user's input_message. If not, say what you still need.",
 'params': {},
 'data_result': {'active_customers': [{'id': 1,
    'name': 'John Doe',
    'email': 'john.doe@example.com',
    'phone': '+1-555-0101',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 2,
    'name': 'Jane Smith',
    'email': 'jane.smith@example.com',
    'phone': '+1-555-0102',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 4,
    'name': 'Alice Williams',
    'email': 'alice.w@techcorp.com',
    'phone': '+1-555-0104',
    'status': 'active',
    'created_at': '2025-12-03 23:40:38',
    'updated_at': '2025-12-03 23:40:38'},
   {'id': 5,
    'name': 'Charlie Brown',
    'email': 'charlie.brown@email.com',
    'phone':

In [58]:
final_answer = m_router_final.get("final_answer")
final_answer

'Based on your request, here are all the active customers who currently have open tickets:\n- John Doe\n- Jane Smith\n- Alice Williams\n- Charlie Brown\n- Diana Prince\n- Edward Norton\n- George Miller\n- Hannah Lee\n- Isaac Newton\n- Julia Roberts\n- Michael Scott'

##FULL PIPELINES IMPLEMENTATION

In [36]:
# ============================================================
# MASTER PIPELINE
# ============================================================

async def run_query(user_message: str) -> str:
    """
    Full automated pipeline:
      1. Always start with Router
      2. Scenario selection
      3. Run correct pipeline
      4. Stop early if final_answer appears
    """

    #-------------------------------------------------------
    # 1. FIRST ROUTER CALL (MANDATORY)
    #-------------------------------------------------------
    print("\n=== PIPELINE START ===")
    state = await a2a_client.create_task(ROUTER_URL, user_message)
    print("ROUTER →", state)

    # Early stop?
    if state.get("final_answer"):
        print("✓ FINAL from Router")
        return state["final_answer"]

    scenario = state.get("scenario")
    print(f"Detected scenario: {scenario}")

    #-------------------------------------------------------
    # 2. ROUTE TO SCENARIO PIPELINE
    #-------------------------------------------------------
    if scenario == "simple":
        state = await run_simple_pipeline(state)

    elif scenario == "negotiation_escalation":
        state = await run_negotiation_pipeline(state)

    elif scenario == "multi_step":
        state = await run_multi_step_pipeline(state)

    else:
        return f"[ERROR] Unknown scenario: {scenario}"

    #-------------------------------------------------------
    # 3. FINAL SAFETY HOP (optional but recommended)
    #-------------------------------------------------------
    if not state.get("final_answer"):
        state = await a2a_client.create_task(ROUTER_URL, state)
        print("FINAL ROUTER HOP →", state)

    return state.get("final_answer") or state


In [37]:
async def run_simple_pipeline(state):
    print("\n--- SIMPLE PIPELINE ---")

    # SUPPORT (step 1)
    state = await a2a_client.create_task(SUPPORT_URL, state)
    print("SUPPORT →", state)
    if state.get("final_answer"):
        return state

    # ROUTER (translate needs_data)
    state = await a2a_client.create_task(ROUTER_URL, state)
    print("ROUTER →", state)
    if state.get("final_answer"):
        return state

    # DATA
    state = await a2a_client.create_task(DATA_URL, state)
    print("DATA →", state)

    # ROUTER again
    state = await a2a_client.create_task(ROUTER_URL, state)
    print("ROUTER →", state)
    if state.get("final_answer"):
        return state

    # SUPPORT final pass
    state = await a2a_client.create_task(SUPPORT_URL, state)
    print("SUPPORT →", state)

    return state


In [38]:
async def run_negotiation_pipeline(state):
    print("\n--- NEGOTIATION PIPELINE ---")

    # SUPPORT (step 1)
    state = await a2a_client.create_task(SUPPORT_URL, state)
    print("SUPPORT →", state)
    if state.get("final_answer"):
        return state

    # ROUTER
    state = await a2a_client.create_task(ROUTER_URL, state)
    print("ROUTER →", state)
    if state.get("final_answer"):
        return state

    # DATA
    state = await a2a_client.create_task(DATA_URL, state)
    print("DATA →", state)

    # ROUTER
    state = await a2a_client.create_task(ROUTER_URL, state)
    print("ROUTER →", state)
    if state.get("final_answer"):
        return state

    # SUPPORT (final)
    state = await a2a_client.create_task(SUPPORT_URL, state)
    print("SUPPORT →", state)

    return state


In [39]:
async def run_multi_step_pipeline(state):
    print("\n--- MULTI-STEP PIPELINE ---")

    # SUPPORT (step 1)
    state = await a2a_client.create_task(SUPPORT_URL, state)
    print("SUPPORT →", state)
    if state.get("final_answer"):
        return state

    # ROUTER
    state = await a2a_client.create_task(ROUTER_URL, state)
    print("ROUTER →", state)
    if state.get("final_answer"):
        return state

    # DATA (active_customers)
    state = await a2a_client.create_task(DATA_URL, state)
    print("DATA →", state)

    # ROUTER
    state = await a2a_client.create_task(ROUTER_URL, state)
    print("ROUTER →", state)
    if state.get("final_answer"):
        return state

    # SUPPORT (ticket_history request)
    state = await a2a_client.create_task(SUPPORT_URL, state)
    print("SUPPORT →", state)
    if state.get("final_answer"):
        return state

    # ROUTER
    state = await a2a_client.create_task(ROUTER_URL, state)
    print("ROUTER →", state)

    # DATA (ticket_history)
    state = await a2a_client.create_task(DATA_URL, state)
    print("DATA →", state)

    # ROUTER
    state = await a2a_client.create_task(ROUTER_URL, state)
    print("ROUTER →", state)
    if state.get("final_answer"):
        return state

    # SUPPORT final
    state = await a2a_client.create_task(SUPPORT_URL, state)
    print("SUPPORT →", state)

    return state


## FINAL AGENT TO AGENT IMPLEMENTATION

In [40]:
answer = await run_query("Get customer information for ID 5")
print(answer)


=== PIPELINE START ===
ROUTER → {'scenario': 'simple', 'input_message': 'Get customer information for ID 5', 'task': "Interpret the user's `input_message` and determine the necessary action. Your output must be a JSON object with a `type` and a `need`.", 'params': {}, 'data_result': None, 'support_response': None, 'final_answer': None}
Detected scenario: simple

--- SIMPLE PIPELINE ---
SUPPORT → {'scenario': 'simple', 'input_message': 'Get customer information for ID 5', 'task': "Interpret the user's `input_message` and determine the necessary action. Your output must be a JSON object with a `type` and a `need`.", 'params': {}, 'data_result': None, 'support_response': {'type': 'needs_data', 'need': 'fetch_customer_info', 'params': {'customer_id': 5}}, 'final_answer': None}
ROUTER → {'scenario': 'simple', 'input_message': 'Get customer information for ID 5', 'task': 'fetch_customer_info', 'params': {'customer_id': 5}, 'data_result': None, 'support_response': {'type': 'needs_data', 'nee

In [41]:
print(answer)

Customer Charlie Brown's account is active. Their email is charlie.brown@email.com.


Note: All the information of charlie is retieved in the output dict, but answer generated is based on the query in the beggining. "Get ALL information from customer ID 5" may be better when one wants to show everything

In [44]:
answer = await run_query("I've been charged twice, please refund immediately!")


=== PIPELINE START ===
ROUTER → {'scenario': 'negotiation_escalation', 'input_message': "I've been charged twice, please refund immediately!", 'task': "Interpret the user's request and identify the necessary next steps or required data.", 'params': {}, 'data_result': None, 'support_response': None, 'final_answer': None}
Detected scenario: negotiation_escalation

--- NEGOTIATION PIPELINE ---
SUPPORT → {'scenario': 'negotiation_escalation', 'input_message': "I've been charged twice, please refund immediately!", 'task': "Interpret the user's request and identify the necessary next steps or required data.", 'params': {}, 'data_result': None, 'support_response': None, 'final_answer': "I'm really sorry to hear about this urgent issue. I want to resolve it as quickly as possible. \nCould you please provide your customer ID so I can look into this immediately?"}


In [45]:
print(answer)

I'm really sorry to hear about this urgent issue. I want to resolve it as quickly as possible. 
Could you please provide your customer ID so I can look into this immediately?


Note: The system observed that: 1. The customer is in an emergency and 2. The customer wants something done but did not provide ID information.

Therefore, it consoles the customer and ask for ID

In [66]:
answer = await run_query("Show me all active customers who have open tickets")


=== PIPELINE START ===
ROUTER → {'scenario': 'multi_step', 'input_message': 'Show me all active customers who have open tickets', 'task': "Interpret the user's `input_message` and determine what data is needed to answer it.", 'params': {}, 'data_result': None, 'support_response': None, 'final_answer': None}
Detected scenario: multi_step

--- MULTI-STEP PIPELINE ---
SUPPORT → {'scenario': 'multi_step', 'input_message': 'Show me all active customers who have open tickets', 'task': "Interpret the user's `input_message` and determine what data is needed to answer it.", 'params': {}, 'data_result': None, 'support_response': {'type': 'needs_data', 'need': 'fetch_active_customers'}, 'final_answer': None}
ROUTER → {'scenario': 'multi_step', 'input_message': 'Show me all active customers who have open tickets', 'task': 'fetch_active_customers', 'params': {}, 'data_result': None, 'support_response': {'type': 'needs_data', 'need': 'fetch_active_customers'}, 'final_answer': None}
DATA → {'scenari

In [46]:
print(answer)

I have found the following active customers with open tickets: John Doe, Jane Smith, Alice Williams, Charlie Brown, Diana Prince, Edward Norton, George Miller, Hannah Lee, Isaac Newton, Julia Roberts, Michael Scott.


## Conclusion

This project gave me a practical understanding of how multi-agent systems coordinate using A2A, and how clearly separating responsibilities across specialized agents leads to stable multi-turn workflows. By building an MCP-backed Data Agent, a reasoning-focused Support Agent, and a Router that orchestrates the entire flow, I learned how agents can communicate through structured JSON while following strict task boundaries. A major part of this project was constructing an MCP server using FastMCP and the MCPToolset from Google ADK. I created custom MCP tools such as get_customer, list_customers, update_customer, create_ticket, and get_customer_history, and registered them inside the MCPToolset so that the Data Agent could call them programmatically. This experience clarified how tool schemas, arguments, and return structures must be designed so an agent can reliably use them without hallucinating parameters.

A significant challenge came from stabilizing multi-step coordination. The Router must detect the scenario, correctly translate Support Agent’s “needs_data” messages into Data Agent tasks, and maintain the full JSON packet without losing fields between turns. Small prompt inconsistencies could cause the Router to stop forwarding intermediate results or overwrite data, which taught me how sensitive LLM-driven orchestration can be. Implementing explicit logging, consistent JSON formatting, and a strict scenario pipeline helped me debug these issues. Overall, this assignment gave me hands-on experience with building custom MCP tools, wiring them to agents through MCPToolset, designing multi-turn A2A flows, and debugging distributed AI workflows—skills that are crucial for real-world multi-agent system design.

## Version Check

In [47]:
# ====================================================
# Comprehensive Environment Version Check
# ====================================================

import importlib
import pkg_resources

libraries = {
    "google-genai": None,
    "google-adk": None,
    "a2a-sdk": None,
    "fastmcp": None,
    "httpx": None,
    "uvicorn": None,
    "aiohttp": None,
    "anyio": None,
    "nest_asyncio": None,
    "python-dotenv": None,
    "requests": None,
}

print("=== Checking Installed Versions ===\n")

for lib in libraries:
    try:
        version = pkg_resources.get_distribution(lib).version
        libraries[lib] = version
        print(f"{lib:<15}  OK   version {version}")
    except Exception as e:
        print(f"{lib:<15}  MISSING  ({e})")

print("\n=== Import Test ===\n")

imports = [
    "google.genai",
    "google.adk",
    "google.adk.agents",
    "a2a.client",
    "a2a.server",
    "fastmcp",
    "httpx",
    "uvicorn",
    "dotenv",
]

for module in imports:
    try:
        importlib.import_module(module)
        print(f"{module:<30}  OK")
    except Exception as e:
        print(f"{module:<30}  FAIL ({e})")


=== Checking Installed Versions ===

google-genai     OK   version 1.53.0
google-adk       OK   version 1.19.0
a2a-sdk          OK   version 0.3.0
fastmcp          OK   version 2.13.3
httpx            OK   version 0.28.1
uvicorn          OK   version 0.38.0
aiohttp          OK   version 3.13.2
anyio            OK   version 4.11.0
nest_asyncio     OK   version 1.6.0
python-dotenv    OK   version 1.2.1
requests         OK   version 2.32.5

=== Import Test ===

google.genai                    OK
google.adk                      OK
google.adk.agents               OK
a2a.client                      OK
a2a.server                      OK
fastmcp                         OK
httpx                           OK
uvicorn                         OK
dotenv                          OK
